In [45]:
import pandas as pd    
import numpy as np
from numpy import reshape
from sklearn import preprocessing as pre

***Put CSV into Dataframe***

In [46]:
df = pd.read_csv('valorant_data.csv')
print(df)

      map_id map_name  total_score  ct_score  t_score  agent_1 agent_2  \
0          3    Lotus           14         9        3  KILLJOY   VIPER   
1          4    Lotus           12         9        3   CYPHER    RAZE   
2          5    cebox            6         2        4    VIPER    OMEN   
3          6    cebox           13         3       10     RAZE  CYPHER   
4          7     Bind           14        10        2     JETT   GEKKO   
...      ...      ...          ...       ...      ...      ...     ...   
3705    3708   Sunset           13         3       10     RAZE  CYPHER   
3706    3709   Sunset           13         3       10     SOVA    RAZE   
3707    3710   Sunset           10         1        9   CYPHER     ISO   
3708    3711    Lotus           13         4        9     SOVA  BREACH   
3709    3712    Lotus            3         3        0      ISO  CYPHER   

      agent_3 agent_4  agent_5  
0        OMEN    RAZE     FADE  
1        OMEN    NEON     FADE  
2     KILLJO

There were a few issues with the scraping, and the map **Icebox** was scraped as **cebox**. So I replaced simply replaced the values.

In [47]:
df.replace(to_replace='cebox', value='Icebox')

,map_id,map_name,total_score,ct_score,t_score,agent_1,agent_2,agent_3,agent_4,agent_5
0,3,Lotus,14,9,3,KILLJOY,VIPER,OMEN,RAZE,FADE
1,4,Lotus,12,9,3,CYPHER,RAZE,OMEN,NEON,FADE
2,5,Icebox,6,2,4,VIPER,OMEN,KILLJOY,FADE,RAZE
3,6,Icebox,13,3,10,RAZE,CYPHER,OMEN,FADE,NEON
4,7,Bind,14,10,2,JETT,GEKKO,VIPER,KAYO,KILLJOY
...,...,...,...,...,...,...,...,...,...,...
3705,3708,Sunset,13,3,10,RAZE,CYPHER,FADE,BREACH,OMEN
3706,3709,Sunset,13,3,10,SOVA,RAZE,BREACH,CYPHER,OMEN
3707,3710,Sunset,10,1,9,CYPHER,ISO,OMEN,NEON,SOVA
3708,3711,Lotus,13,4,9,SOVA,BREACH,RAZE,OMEN,CYPHER


In [48]:
le = pre.LabelEncoder()
df['map_name_en'] = le.fit_transform(df["map_name"])
df = df.drop(columns='map_name')

In [49]:
le.fit(df['agent_1'])
df['agent1'] = le.transform(df["agent_1"])
df['agent2'] = le.transform(df["agent_2"])
df['agent3'] = le.transform(df["agent_3"])
df['agent4'] = le.transform(df["agent_4"]) 
df['agent5'] = le.transform(df["agent_5"])

df = df.drop(columns=["agent_1", "agent_2", "agent_3", "agent_4", "agent_5"])

In [50]:
df = df.rename(columns={'map_name_en' : 'map'})

In [51]:
for x in range(1,23):
    df[f'agent_{x}'] = df[['agent1', 'agent2', 'agent3', 'agent4', 'agent5']].apply(lambda row: 1 if x in row.values else 0, axis = 1)

In [52]:
df = df.drop(columns=['agent1', 'agent2', 'agent3', 'agent4', 'agent5'])

In [53]:
df = df.drop(columns=['map_id'])

In [54]:
df['win'] = df['total_score'].apply(lambda score: 1 if score >= 13 else 0)

In [55]:
df = df.drop(columns = 'total_score')

In [56]:
df['ctscore'] = df['ct_score'].apply(lambda row: 1 if row > 6 else(.5 if row == 6 else 0))
df['tscore'] = df['t_score'].apply(lambda row: 1 if row > 6 else(.5 if row == 6 else 0))

In [57]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split


In [58]:
model = RandomForestClassifier(random_state=60)

In [59]:
X = df[['map'] + [f'agent_{x}' for x in range(1, 23)] + ['tscore', 'ctscore']]
y = df['win']

In [60]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [61]:
X

,map,agent_1,agent_2,agent_3,agent_4,agent_5,agent_6,agent_7,agent_8,agent_9,...,agent_15,agent_16,agent_17,agent_18,agent_19,agent_20,agent_21,agent_22,tscore,ctscore
0,5,0,0,0,0,0,0,1,0,0,...,1,0,1,0,0,0,0,1,0.0,1.0
1,5,0,0,0,0,1,0,1,0,0,...,1,0,1,0,0,0,0,0,0.0,1.0
2,8,0,0,0,0,0,0,1,0,0,...,1,0,1,0,0,0,0,1,0.0,0.0
3,8,0,0,0,0,1,0,1,0,0,...,1,0,1,0,0,0,0,0,1.0,0.0
4,2,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3705,7,1,0,0,0,1,0,1,0,0,...,1,0,1,0,0,0,0,0,1.0,0.0
3706,7,1,0,0,0,1,0,0,0,0,...,1,0,1,0,0,0,1,0,1.0,0.0
3707,7,0,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,1,0,1.0,0.0
3708,5,1,0,0,0,1,0,0,0,0,...,1,0,1,0,0,0,1,0,1.0,0.0


In [62]:
model.fit(X_train, y_train)

RandomForestClassifier(random_state=60)

In [63]:
from sklearn.metrics import accuracy_score

In [64]:
y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)

0.8710204081632653

In [65]:
import joblib

joblib.dump(model, "model.joblib")

['model.joblib']